## Imports

In [ ]:
# Import necessary libraries for this example
from PIL import Image
import numpy as np

from IPython.display import display, Math

# Locate PsPolypy. Only necessary if PsPolypy is not installed as a package.
import sys
sys.path.append('../src/')

# Import PsPolypy
import  PsPolypy

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
from cycler import cycler

# rcParams settings

plt.rcParams['axes.prop_cycle'] = cycler(color='krbmgcy')

plt.rcParams['axes.linewidth'] = 1.5

plt.rcParams['xtick.top'] = plt.rcParams['ytick.right'] = True
plt.rcParams['xtick.minor.visible'] = plt.rcParams['ytick.minor.visible'] = True
plt.rcParams['xtick.direction'] = plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['xtick.major.size'] = plt.rcParams['ytick.major.size'] = 8 
plt.rcParams['xtick.minor.size'] = plt.rcParams['ytick.minor.size'] = 5
plt.rcParams['xtick.major.width'] = plt.rcParams['ytick.major.width'] = 1; 
plt.rcParams['savefig.bbox']='tight'
plt.rcParams['savefig.transparent']=True

plt.rcParams['legend.framealpha'] = 0.6
plt.rcParams['legend.markerscale'] = 2.
plt.rcParams['legend.fontsize'] = 'xx-large'
plt.rcParams['figure.subplot.wspace'] = 0.02
plt.rcParams['figure.subplot.hspace'] = 0.02
plt.rcParams['axes.labelpad'] = 5.
plt.rcParams['figure.dpi'] = 300

plt.rcParams['xtick.labelsize'] = plt.rcParams['ytick.labelsize'] = 20
plt.rcParams['axes.labelsize'] = 24
plt.rcParams['figure.figsize'] = (8, 6)

plt.rcParams["font.family"]='serif'
plt.rcParams["mathtext.fontset"]='stix';

In [ ]:
# Set the noise standard deviations for the noisy images.
noise_stds = [0.01, 0.02, 0.03, 0.04, 0.05]

# Create an instance of the Polydat class from a list of numpy arrays.
filepaths = ['example_images/exampleCL0.png', 'example_images/exampleCL1.png']
raw_imageset = []
for filepath in filepaths:
    with Image.open(filepath) as img:
        grayscale = img.convert('L')
        image_data = np.array(grayscale) / 255.0
        raw_imageset.append(image_data)

np.random.seed(0)
# Create a list of noisy images with different standard deviations.
noisy_imagesets = []
for std in noise_stds:
    noised_set = [image + np.random.normal(0, std, image.shape) for image in raw_imageset]
    noisy_imagesets.append(noised_set)

In [ ]:
raw_polydat = PsPolypy.Polymer.Polydat(raw_imageset, resolution = 2)

# Upscale the image by a factor of 2 using bi-cubic interpolation.
raw_polydat.upscale(magnification = 2, order = 3)

# Segment the particles in the image.
raw_polydat.segment_particles()

# Skeletonize the particles.
raw_polydat.skeletonize_particles()

# Classify the particles.
raw_polydat.classify_particles()

# Filter the particles to only include the Linear classification.
raw_polydat.filter_particles(classifications = ['Linear'])

# Interpolate the skeletons of the particles.
raw_polydat.interpolate_skeletons(step_size = 0.5, k = 3, s = .5)

# Calculate <R^2>
raw_polydat.calc_displacements()

L_min = 10
L_max = 45
# Calculate the persistence length.
raw_polydat.calc_R2_lp(lp_init = 11, min_fitting_length = L_min, max_fitting_length = L_max, fit_kwargs = {'scale_covar': True})



# Print a summary of the polydat object.
raw_polydat.print_summary()

# Create a figure containing the complete workflow plots.
fig_R2, ax = plt.subplots(2,1, figsize = (8,8),sharex=True)

# Plot contour length distribution
ax[0] = raw_polydat.plot_contour_distribution(ax = ax[0], n_points = 1000,
                                          inc_dist_kwargs = {'color': 'Blue', 'lw': 2, 'label': 'Included Distribution'},
                                          inc_fill_kwargs = {'color': 'LightBlue', 'alpha': 0.5},
                                          exc_dist_kwargs = {'color': 'Gray', 'lw': 2, 'alpha': 0.5, 'label': 'Excluded Distribution'},
                                          exc_fill_kwargs = {'color': 'LightGray', 'alpha': 0.5},
                                          vline_kwargs = {'color': 'Blue', 'lw': 0.75, 'dashes': [8,3]})

ax[0].set_ylim(0,0.065)
ax[0].set_ylabel(r'$P(L)$',fontsize=18)
ax[0].grid(lw=0.3)
ax[0].legend(fontsize = 14)

# Plot tangent-tangent correlation with the fit.
raw_polydat.plot_mean_squared_displacements(ax = ax[1], error_bars = True, )
                                                # inc_kwargs = {'color': 'Blue', 'fmt': '.', 'ecolor': 'LightBlue', 'label': 'Fitted Data'},
                                                # exc_kwargs = {'color': 'Gray', 'fmt': '.', 'ecolor': 'LightGray', 'label': 'Excluded Data'})
raw_polydat.plot_mean_squared_displacements_fit(ax = ax[1], show_init = False, show_ci = True, show_pd = True,)
                                                    # fit_kwargs = {'color': 'Red', 'lw': 1.5, 'label': 'Best Fit'},
                                                    # init_kwargs = {'color': 'Red', 'lw': 0.75, 'linestyle': '--', 'label': 'Initial Guess'})
ax[1].set_xlim([0,50])
ax[1].set_ylim(0,2200)
ax[1].set_xlabel(rf'$L\;[\times{raw_polydat.resolution:.0f} nm]$',fontsize=18)
ax[1].set_ylabel(r'$\langle R^2(L)\rangle$',fontsize=18)


ax[1].grid(lw=0.3)
ax[1].legend(fontsize = 14)

# Display the plot.
plt.show()
# Show the fitting results.
# polydat.wlc_fit_result

result = raw_polydat.R2_fit_result
# Extract best-fit parameter and its standard deviation
lp_r2 = result.params['lp'].value
std_r2 = result.params['lp'].stderr

# Goodness-of-fit measure
rchi2_r2 = result.redchi  # Reduced chi-squared
display(Math(rf"$χ_{{red}}^2 = {rchi2_r2:.2f}"))
print('-'*32)
display(Math(rf"$l_p = {lp_r2:.1f} ± {np.sqrt(rchi2_r2)*std_r2:.1f}$"))

In [ ]:
polydat = PsPolypy.Polymer.Polydat(noisy_imagesets[0], resolution = 2)

# Upscale the image by a factor of 2 using bi-cubic interpolation.
polydat.upscale(magnification = 2, order = 3)

# Segment the particles in the image.
polydat.segment_particles()

# Skeletonize the particles.
polydat.skeletonize_particles()

# Classify the particles.
polydat.classify_particles()

# Filter the particles to only include the Linear classification.
polydat.filter_particles(classifications = ['Linear'])

# Interpolate the skeletons of the particles.
polydat.interpolate_skeletons(step_size = 0.5, k = 3, s = .5)

# Calculate <R^2>
polydat.calc_displacements()

L_min = 10
L_max = 45
# Calculate the persistence length.
polydat.calc_R2_lp(lp_init = 11, min_fitting_length = L_min, max_fitting_length = L_max, fit_kwargs = {'scale_covar': True})

# Print a summary of the polydat object.
polydat.print_summary()

# Create a figure containing the complete workflow plots.
fig_R2, ax = plt.subplots(2,1, figsize = (8,8),sharex=True)

# Plot contour length distribution
ax[0] = polydat.plot_contour_distribution(ax = ax[0], n_points = 1000,
                                          inc_dist_kwargs = {'color': 'Blue', 'lw': 2, 'label': 'Included Distribution'},
                                          inc_fill_kwargs = {'color': 'LightBlue', 'alpha': 0.5},
                                          exc_dist_kwargs = {'color': 'Gray', 'lw': 2, 'alpha': 0.5, 'label': 'Excluded Distribution'},
                                          exc_fill_kwargs = {'color': 'LightGray', 'alpha': 0.5},
                                          vline_kwargs = {'color': 'Blue', 'lw': 0.75, 'dashes': [8,3]})

ax[0].set_ylim(0,0.065)
ax[0].set_ylabel(r'$P(L)$',fontsize=18)
ax[0].grid(lw=0.3)
ax[0].legend(fontsize = 14)

# Plot tangent-tangent correlation with the fit.
polydat.plot_mean_squared_displacements(ax = ax[1], error_bars = True, )
                                                # inc_kwargs = {'color': 'Blue', 'fmt': '.', 'ecolor': 'LightBlue', 'label': 'Fitted Data'},
                                                # exc_kwargs = {'color': 'Gray', 'fmt': '.', 'ecolor': 'LightGray', 'label': 'Excluded Data'})
polydat.plot_mean_squared_displacements_fit(ax = ax[1], show_init = False, show_ci = True, show_pd = True,)
                                                    # fit_kwargs = {'color': 'Red', 'lw': 1.5, 'label': 'Best Fit'},
                                                    # init_kwargs = {'color': 'Red', 'lw': 0.75, 'linestyle': '--', 'label': 'Initial Guess'})
ax[1].set_xlim([0,50])
ax[1].set_ylim(0,2200)
ax[1].set_xlabel(rf'$L\;[\times{polydat.resolution:.0f} nm]$',fontsize=18)
ax[1].set_ylabel(r'$\langle R^2(L)\rangle$',fontsize=18)


ax[1].grid(lw=0.3)
ax[1].legend(fontsize = 14)

# Display the plot.
plt.show()
# Show the fitting results.
# polydat.wlc_fit_result

result = polydat.R2_fit_result
# Extract best-fit parameter and its standard deviation
lp_r2 = result.params['lp'].value
std_r2 = result.params['lp'].stderr

# Goodness-of-fit measure
rchi2_r2 = result.redchi  # Reduced chi-squared
display(Math(rf"$χ_{{red}}^2 = {rchi2_r2:.2f}"))
print('-'*32)
display(Math(rf"$l_p = {lp_r2:.1f} ± {np.sqrt(rchi2_r2)*std_r2:.1f}$"))

In [ ]:
polydat = PsPolypy.Polymer.Polydat(noisy_imagesets[1], resolution = 2)

# Upscale the image by a factor of 2 using bi-cubic interpolation.
polydat.upscale(magnification = 2, order = 3)

# Segment the particles in the image.
polydat.segment_particles()

# Skeletonize the particles.
polydat.skeletonize_particles()

# Classify the particles.
polydat.classify_particles()

# Filter the particles to only include the Linear classification.
polydat.filter_particles(classifications = ['Linear'])

# Interpolate the skeletons of the particles.
polydat.interpolate_skeletons(step_size = 0.5, k = 3, s = .5)

# Calculate <R^2>
polydat.calc_displacements()

L_min = 11
L_max = 41
# Calculate the persistence length.
polydat.calc_R2_lp(lp_init = 11, min_fitting_length = L_min, max_fitting_length = L_max, fit_kwargs = {'scale_covar': True})

# Print a summary of the polydat object.
polydat.print_summary()

# Create a figure containing the complete workflow plots.
fig_R2, ax = plt.subplots(2,1, figsize = (8,8),sharex=True)

# Plot contour length distribution
ax[0] = polydat.plot_contour_distribution(ax = ax[0], n_points = 1000,
                                          inc_dist_kwargs = {'color': 'Blue', 'lw': 2, 'label': 'Included Distribution'},
                                          inc_fill_kwargs = {'color': 'LightBlue', 'alpha': 0.5},
                                          exc_dist_kwargs = {'color': 'Gray', 'lw': 2, 'alpha': 0.5, 'label': 'Excluded Distribution'},
                                          exc_fill_kwargs = {'color': 'LightGray', 'alpha': 0.5},
                                          vline_kwargs = {'color': 'Blue', 'lw': 0.75, 'dashes': [8,3]})

ax[0].set_ylim(0,0.065)
ax[0].set_ylabel(r'$P(L)$',fontsize=18)
ax[0].grid(lw=0.3)
ax[0].legend(fontsize = 14)

# Plot tangent-tangent correlation with the fit.
polydat.plot_mean_squared_displacements(ax = ax[1], error_bars = True, )
                                                # inc_kwargs = {'color': 'Blue', 'fmt': '.', 'ecolor': 'LightBlue', 'label': 'Fitted Data'},
                                                # exc_kwargs = {'color': 'Gray', 'fmt': '.', 'ecolor': 'LightGray', 'label': 'Excluded Data'})
polydat.plot_mean_squared_displacements_fit(ax = ax[1], show_init = False, show_ci = True, show_pd = True,)
                                                    # fit_kwargs = {'color': 'Red', 'lw': 1.5, 'label': 'Best Fit'},
                                                    # init_kwargs = {'color': 'Red', 'lw': 0.75, 'linestyle': '--', 'label': 'Initial Guess'})
ax[1].set_xlim([0,50])
ax[1].set_ylim(0,2200)
ax[1].set_xlabel(rf'$L\;[\times{polydat.resolution:.0f} nm]$',fontsize=18)
ax[1].set_ylabel(r'$\langle R^2(L)\rangle$',fontsize=18)


ax[1].grid(lw=0.3)
ax[1].legend(fontsize = 14)

# Display the plot.
plt.show()
# Show the fitting results.
# polydat.wlc_fit_result

result = polydat.R2_fit_result
# Extract best-fit parameter and its standard deviation
lp_r2 = result.params['lp'].value
std_r2 = result.params['lp'].stderr

# Goodness-of-fit measure
rchi2_r2 = result.redchi  # Reduced chi-squared
display(Math(rf"$χ_{{red}}^2 = {rchi2_r2:.2f}"))
print('-'*32)
display(Math(rf"$l_p = {lp_r2:.1f} ± {np.sqrt(rchi2_r2)*std_r2:.1f}$"))

In [ ]:
polydat = PsPolypy.Polymer.Polydat(noisy_imagesets[2], resolution = 2)

# Upscale the image by a factor of 2 using bi-cubic interpolation.
polydat.upscale(magnification = 2, order = 3)

# Segment the particles in the image.
polydat.segment_particles()

# Skeletonize the particles.
polydat.skeletonize_particles()

# Classify the particles.
polydat.classify_particles()

# Filter the particles to only include the Linear classification.
polydat.filter_particles(classifications = ['Linear'])

# Interpolate the skeletons of the particles.
polydat.interpolate_skeletons(step_size = 0.5, k = 3, s = .5)

# Calculate <R^2>
polydat.calc_displacements()

L_min = 11
L_max = 35
# Calculate the persistence length.
polydat.calc_R2_lp(lp_init = 11, min_fitting_length = L_min, max_fitting_length = L_max, fit_kwargs = {'scale_covar': True})

# Print a summary of the polydat object.
polydat.print_summary()

# Create a figure containing the complete workflow plots.
fig_R2, ax = plt.subplots(2,1, figsize = (8,8),sharex=True)

# Plot contour length distribution
ax[0] = polydat.plot_contour_distribution(ax = ax[0], n_points = 1000,
                                          inc_dist_kwargs = {'color': 'Blue', 'lw': 2, 'label': 'Included Distribution'},
                                          inc_fill_kwargs = {'color': 'LightBlue', 'alpha': 0.5},
                                          exc_dist_kwargs = {'color': 'Gray', 'lw': 2, 'alpha': 0.5, 'label': 'Excluded Distribution'},
                                          exc_fill_kwargs = {'color': 'LightGray', 'alpha': 0.5},
                                          vline_kwargs = {'color': 'Blue', 'lw': 0.75, 'dashes': [8,3]})

ax[0].set_ylim(0,0.065)
ax[0].set_ylabel(r'$P(L)$',fontsize=18)
ax[0].grid(lw=0.3)
ax[0].legend(fontsize = 14)

# Plot tangent-tangent correlation with the fit.
polydat.plot_mean_squared_displacements(ax = ax[1], error_bars = True, )
                                                # inc_kwargs = {'color': 'Blue', 'fmt': '.', 'ecolor': 'LightBlue', 'label': 'Fitted Data'},
                                                # exc_kwargs = {'color': 'Gray', 'fmt': '.', 'ecolor': 'LightGray', 'label': 'Excluded Data'})
polydat.plot_mean_squared_displacements_fit(ax = ax[1], show_init = False, show_ci = True, show_pd = True,)
                                                    # fit_kwargs = {'color': 'Red', 'lw': 1.5, 'label': 'Best Fit'},
                                                    # init_kwargs = {'color': 'Red', 'lw': 0.75, 'linestyle': '--', 'label': 'Initial Guess'})
ax[1].set_xlim([0,50])
ax[1].set_ylim(0,2200)
ax[1].set_xlabel(rf'$L\;[\times{polydat.resolution:.0f} nm]$',fontsize=18)
ax[1].set_ylabel(r'$\langle R^2(L)\rangle$',fontsize=18)


ax[1].grid(lw=0.3)
ax[1].legend(fontsize = 14)

# Display the plot.
plt.show()
# Show the fitting results.
# polydat.wlc_fit_result

result = polydat.R2_fit_result
# Extract best-fit parameter and its standard deviation
lp_r2 = result.params['lp'].value
std_r2 = result.params['lp'].stderr

# Goodness-of-fit measure
rchi2_r2 = result.redchi  # Reduced chi-squared
display(Math(rf"$χ_{{red}}^2 = {rchi2_r2:.2f}"))
print('-'*32)
display(Math(rf"$l_p = {lp_r2:.1f} ± {np.sqrt(rchi2_r2)*std_r2:.1f}$"))

In [ ]:
polydat = PsPolypy.Polymer.Polydat(noisy_imagesets[3], resolution = 2)

# Upscale the image by a factor of 2 using bi-cubic interpolation.
polydat.upscale(magnification = 2, order = 3)

# Segment the particles in the image.
polydat.segment_particles()

# Skeletonize the particles.
polydat.skeletonize_particles()

# Classify the particles.
polydat.classify_particles()

# Filter the particles to only include the Linear classification.
polydat.filter_particles(classifications = ['Linear'])

# Interpolate the skeletons of the particles.
polydat.interpolate_skeletons(step_size = 0.5, k = 3, s = .5)

# Calculate <R^2>
polydat.calc_displacements()

L_min = 6
L_max = 35
# Calculate the persistence length.
polydat.calc_R2_lp(lp_init = 11, min_fitting_length = L_min, max_fitting_length = L_max, fit_kwargs = {'scale_covar': True})

# Print a summary of the polydat object.
polydat.print_summary()

# Create a figure containing the complete workflow plots.
fig_R2, ax = plt.subplots(2,1, figsize = (8,8),sharex=True)

# Plot contour length distribution
ax[0] = polydat.plot_contour_distribution(ax = ax[0], n_points = 1000,
                                          inc_dist_kwargs = {'color': 'Blue', 'lw': 2, 'label': 'Included Distribution'},
                                          inc_fill_kwargs = {'color': 'LightBlue', 'alpha': 0.5},
                                          exc_dist_kwargs = {'color': 'Gray', 'lw': 2, 'alpha': 0.5, 'label': 'Excluded Distribution'},
                                          exc_fill_kwargs = {'color': 'LightGray', 'alpha': 0.5},
                                          vline_kwargs = {'color': 'Blue', 'lw': 0.75, 'dashes': [8,3]})

ax[0].set_ylim(0,0.09)
ax[0].set_ylabel(r'$P(L)$',fontsize=18)
ax[0].grid(lw=0.3)
ax[0].legend(fontsize = 14)

# Plot tangent-tangent correlation with the fit.
polydat.plot_mean_squared_displacements(ax = ax[1], error_bars = True, )
                                                # inc_kwargs = {'color': 'Blue', 'fmt': '.', 'ecolor': 'LightBlue', 'label': 'Fitted Data'},
                                                # exc_kwargs = {'color': 'Gray', 'fmt': '.', 'ecolor': 'LightGray', 'label': 'Excluded Data'})
polydat.plot_mean_squared_displacements_fit(ax = ax[1], show_init = False, show_ci = True, show_pd = True,)
                                                    # fit_kwargs = {'color': 'Red', 'lw': 1.5, 'label': 'Best Fit'},
                                                    # init_kwargs = {'color': 'Red', 'lw': 0.75, 'linestyle': '--', 'label': 'Initial Guess'})
ax[1].set_xlim([0,50])
ax[1].set_ylim(0,2200)
ax[1].set_xlabel(rf'$L\;[\times{polydat.resolution:.0f} nm]$',fontsize=18)
ax[1].set_ylabel(r'$\langle R^2(L)\rangle$',fontsize=18)


ax[1].grid(lw=0.3)
ax[1].legend(fontsize = 14)

# Display the plot.
plt.show()
# Show the fitting results.
# polydat.wlc_fit_result

result = polydat.R2_fit_result
# Extract best-fit parameter and its standard deviation
lp_r2 = result.params['lp'].value
std_r2 = result.params['lp'].stderr

# Goodness-of-fit measure
rchi2_r2 = result.redchi  # Reduced chi-squared
display(Math(rf"$χ_{{red}}^2 = {rchi2_r2:.2f}"))
print('-'*32)
display(Math(rf"$l_p = {lp_r2:.1f} ± {np.sqrt(rchi2_r2)*std_r2:.1f}$"))

In [ ]:
polydat = PsPolypy.Polymer.Polydat(noisy_imagesets[4], resolution = 2)

# Upscale the image by a factor of 2 using bi-cubic interpolation.
polydat.upscale(magnification = 2, order = 3)

# Segment the particles in the image.
polydat.segment_particles()

# Skeletonize the particles.
polydat.skeletonize_particles()

# Classify the particles.
polydat.classify_particles()

# Filter the particles to only include the Linear classification.
polydat.filter_particles(classifications = ['Linear'])

# Interpolate the skeletons of the particles.
polydat.interpolate_skeletons(step_size = 0.5, k = 3, s = .5)

# Calculate <R^2>
polydat.calc_displacements()

L_min = 6
L_max = 31
# Calculate the persistence length.
polydat.calc_R2_lp(lp_init = 11, min_fitting_length = L_min, max_fitting_length = L_max, fit_kwargs = {'scale_covar': True})

# Print a summary of the polydat object.
polydat.print_summary()

# Create a figure containing the complete workflow plots.
fig_R2, ax = plt.subplots(2,1, figsize = (8,8),sharex=True)

# Plot contour length distribution
ax[0] = polydat.plot_contour_distribution(ax = ax[0], n_points = 1000,
                                          inc_dist_kwargs = {'color': 'Blue', 'lw': 2, 'label': 'Included Distribution'},
                                          inc_fill_kwargs = {'color': 'LightBlue', 'alpha': 0.5},
                                          exc_dist_kwargs = {'color': 'Gray', 'lw': 2, 'alpha': 0.5, 'label': 'Excluded Distribution'},
                                          exc_fill_kwargs = {'color': 'LightGray', 'alpha': 0.5},
                                          vline_kwargs = {'color': 'Blue', 'lw': 0.75, 'dashes': [8,3]})

ax[0].set_ylim(0,0.11)
ax[0].set_ylabel(r'$P(L)$',fontsize=18)
ax[0].grid(lw=0.3)
ax[0].legend(fontsize = 14)

# Plot tangent-tangent correlation with the fit.
polydat.plot_mean_squared_displacements(ax = ax[1], error_bars = True, )
                                                # inc_kwargs = {'color': 'Blue', 'fmt': '.', 'ecolor': 'LightBlue', 'label': 'Fitted Data'},
                                                # exc_kwargs = {'color': 'Gray', 'fmt': '.', 'ecolor': 'LightGray', 'label': 'Excluded Data'})
polydat.plot_mean_squared_displacements_fit(ax = ax[1], show_init = False, show_ci = True, show_pd = True,)
                                                    # fit_kwargs = {'color': 'Red', 'lw': 1.5, 'label': 'Best Fit'},
                                                    # init_kwargs = {'color': 'Red', 'lw': 0.75, 'linestyle': '--', 'label': 'Initial Guess'})
ax[1].set_xlim([0,50])
ax[1].set_ylim(0,2200)
ax[1].set_xlabel(rf'$L\;[\times{polydat.resolution:.0f} nm]$',fontsize=18)
ax[1].set_ylabel(r'$\langle R^2(L)\rangle$',fontsize=18)


ax[1].grid(lw=0.3)
ax[1].legend(fontsize = 14)

# Display the plot.
plt.show()
# Show the fitting results.
# polydat.wlc_fit_result

result = polydat.R2_fit_result
# Extract best-fit parameter and its standard deviation
lp_r2 = result.params['lp'].value
std_r2 = result.params['lp'].stderr

# Goodness-of-fit measure
rchi2_r2 = result.redchi  # Reduced chi-squared
display(Math(rf"$χ_{{red}}^2 = {rchi2_r2:.2f}"))
print('-'*32)
display(Math(rf"$l_p = {lp_r2:.1f} ± {np.sqrt(rchi2_r2)*std_r2:.1f}$"))

In [ ]:
std = [0, 0.1, 0.2, 0.3, 0.4, 0.5]
lps = [12.5, 12.4, 12.6, 12.6, 12.9, 3.6]
lps_e = [0.4, 0.4, 0.5, 0.2, 4.7, 0.1]

fig, ax = plt.subplots(1,1, figsize = (8,6))
ax.errorbar(std, lps, yerr = lps_e, fmt = 'o', color = 'Blue', label = 'Estimated $l_p$')

